# Lab 5: Training a Transformer

In today's lab session we will:

1. [Build a Simple Transformer Model](#Build-a-Simple-Transformer-Model)
2. [Expand and Compare to a More Complex Transformer Model](#Expanding-to-a-More-Complex-Transformer-Model)
3. [Adding a Classification Token (CLS)](#Adding-a-Classification-Token)
4. [Adding Positional Embeddings](#Adding-Positional-Embeddings)

As a note, this lab is more hands off than previous labs - the transformer code is _relatively_ simple due to the in-built functions in PyTorch so most of this labsheet (and lab) is directed at your ability to train a model and compare findings. You can use all that you have learnt over the past 4 labs to apply here - think of this lab as the final boss of the formative Appied Deep Learning exercises.

As always, if you get stuck or are unsure about something, please ask the lecturers and/or the teaching assistants.

---
# Build a Simple Transformer Model

For the whole lab, we will focus on recreating a simpler version of the VIT architecture which can be seen below:

<center><img alt="VIT Transformer Diagram" src="./media/vit_ex.png" style="max-height: 300px;"></center>

Initially we'll be building the simplest Transformer network of a single encoder layer and a single FC layer for classification.

From the figure, you can see that the image is split into patches, which are small regions of the image that are fed through the transformer model. This allows for relational modelling between different parts of the image.

The first step is to implement this 'patchifying process'...

## Patchifying an Image

The input shape for our images are of size $C \times H \times W$, during training these will be represented by a 4-$D$ tensor which also includes the batch size ($B \times C \times H \times W$).

We want to break our image into $n \times n$ patches. **Note** that $n$ should divide evenly into $H$ and $W$.

**TASK 1:** Implement the patchifying function to convert your input tensor into the form of $B \times ((H \times W)/n^2) \times (n^2 \times C)$. **Note** that this is 3-dimensional.
Look at [`nn.Unfold`](https://pytorch.org/docs/stable/generated/torch.nn.Unfold.html) to do this simply. This operation groups together inputs across a specific kernel size for 4-$D$ inputs. *Hint:* kernel size is your $n$, think back to the definition of pooling layers for your stride value.

Check that your patchify function works with $n=8$, which we will be using for most of this labsheet.

## A Simple Transformer Encoder

For now we are going to skip the linear projection of flattened patches - generally this is to reduce the embedding size so the fully connected layers within the transformer don't grow too big. Instead we're going to start looking at the transformer encoder. Specifically, we will be creating a model with one Transformer Encoder Layer and one FC layer for the classification (the MLP head in the image). An example of the first transformer model can be seen below:

<center><img alt="Step 1 Transformer Diagram" src="./media/step_1.png" style="max-height: 500px;"></center>

Recall from the lectures that Transformers have both an encoder and a decoder section, but we are doing classification instead of reconstruction so we do not need a decoder. With this in mind lets look at the PyTorch implementations, we'll highlight three different structures that are available:

* [`nn.Transformer`](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html#torch.nn.Transformer): This allows you to create an entire Transformer model with both encoders and decoders (each having differing numbers of encoder/decoder layers). As we are focusing only on an encoder we will skip this.
* [`nn.TransformerEncoder`](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoder.html): This allows you to create only the encoder portion of the transformer which can consist of multiple encoder layers.
* [`nn.TransformerEncoderLayer`](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html): This creates only a single encoder layer in the transformer - the smallest atomic unit. This still contains both the Multi-Head Self Attention and the MLP layers which collectively make up *one layer* as well as the activation functions and layer norms (you can think of this as a much higher level than previous labs where we have to define all of these functions!)



The `TransformerEncoderLayer` has a number of arguments, but we only need to worry about three for now: `d_model`, `n_heads`, and `batch_first`. The first specifies the embedding size of the input and the internal layers; the second gives the number of heads in the model; and the third specifies whether the input is (Batch, Sequence, Feature) (Sequence, Batch, Feature).

* As we're starting simple, set `n_heads=1` so that we only have a single head in our self-attention layer - we can explore how the number of heads change the performance later.
* The value of `d_model` needs to be the size of your patches ($n^2 \times C=192$).
* As the dataset returns the data in the form (Batch, Sequence, Feature) we need to set `batch_first` to `True` (this is `False` by default!)

**TASK 2**: Add the transformer encoder layer in the constructor for the `CIFAR_Transformer` class and add it to the forward method. Remember to patchify your input using the result of task 1 before passing this into your transformer.

Run your code on BC4 as a test, printing out the shape of the output of the model after one batch - your program won't fully run yet but this is fine!

<details>
  <summary>Question: What is the expected output shape from the transformer?</summary>
  
  **Answer:** This should be $128 \times 16 \times 192$ as the transformer encoder layer keeps the same dimensionality of the features throughout.
</details>


With this implemented we can start to think of the output and how to turn this into the logits we need for classification. After the output of the transformer our output is too large to pass into a fully connected layer. Most methods use a specific input token which we'll explore later, but for now we are going to average across all outputs (known as global average pooling) for an example and then pass this into the MLP head.

**TASK 3.1:** Add in a fully connected layer into the constructor of `CIFAR_Transformer` which projects from the dimension of your transformer (192) to the number of classes (10). 

**TASK 3.2:** Average using mean the output of the Transformer Encoder before passing this into the fully connected layer - pay attention to which dimension to average over! After this, the forward method should return logits that can be used for classification.

<details>
  <summary>Hint</summary>
  
  You will need to average along the middle dimension of the output of the transformer encoder layer, then this can be passed straight into the fully connected layers. Look at Lab 1 or Lab 2 code if you need a refresher on how to do this.
</details>

Run your code on BC4 and check the output of the forward pass is of size $B \times 10$ where $B$ is your batch size (128 by default).

**TASK 4:** Remove the `sys.exit()` command in the training loop and now you can fully train your transformer model.


The final thing we'll look at before creating a more complex transformer is looking at the number of parameters within the model. We can do this with the following code:

```python
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
```

**TASK 5:** Add count_parameters into your `train_cifar.py` script at the start of your traininer and then compare the number of parameters in your transformer model to the CNN model that we trained in Lab 2.

How does your accuracy compare with the CNN model with the differing number of parameters? Discuss in pairs and/or with a Lecturer/Teaching Assistant as to your findings.

---

# Expanding to a More Complex Transformer Model

We will now expand to a more complex transformer model, namely by adding in extra heads to the MHSA (as until now we have only been using Self Attention!), multiple encoder layers and an MLP for the classification head. The goal for this part is to create the transformer below:

<center><img alt="Step 2 Transformer Diagram" src="./media/step_2.png" style="max-height: 500px;"></center>

## Increasing the Number of Heads

The first and easiest thing to modify is to increase the number of heads within the self attention of the transformer model. You may have noticed one of the input arguments to the script is `args.num_heads`.

**TASK 6:** Run your program comparing using differing numbers of heads. How does increasing the number of heads effect the performance of the model? 
<details>
  <summary>Hint</summary>

  If you want an idea of what heads to run, it is normal to ablate in powers of 2, so try 1, 2, 4, 8 as a starting point.
</details>

## Increasing the Size of the MLP Head

As of now we have used only a single FC layer to project the output of the transformer encoder to the logits for classification, we will now add an extra layer to allow a more complex projection and *hopefully* increase performance!

**TASK 7:** Add an extra fully connected layer to the classification head in both the constructor and the forward method. We suggest a hidden layer size of 100. In pairs and/or with a teacher discuss how this effects performance and number of parameters in the model.

## Increasing the Number of Encoder Layers

Another aspect of increasing the complexity of the model is to increase the number of encoder layers within the model. It is common nowadays for transformer models to have 6 or 8 layers within them, but due to CIFAR-10's complexity, we'll only compare with adding an extra 1 or 2 layers.

**TASK 8:** Exploring adding an extra 1 or 2 encoder layers into the constructor and forward method to see how it effects the performance and number of parameters.

<details>
  <summary>Hint</summary>

Look up [`nn.modulelist`](https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html) which can allow for a variable number of modules within a Deep Learning model and save you hard-coding specific numbers of layers.
</details>

## Ablation Studies

We now have a large number of different model setups and types. From our base model we can use: $[1, 2, 4, 8]$ heads; $[1, 2]$ fully connected layers; and $[1, 2, 3]$ encoder layers. This doesn't even include different size patches, we could also try using $n=4$ or $n=16$ to see how this effects performance (we recommend finishing the rest of this labsheet before running these experiments though!)

If we want to perform a grid search, i.e. searching exhaustively through all options, this explodes exponentially with every new setup, we would have to run $4 \times 2 \times 3 \times 3 = 72$ different experiments!

Instead, to find the best performance it is common to build this iteratively using best case performance. For example, we run $4$ experiments to find the best number of heads, then fix that value and then run $2$ experiments for the fully connected layers and fix that, etc.

In this way, we reduce the number of experiments from $72$ to $12$ which is a lot, but much more manageable.

---
# Adding a Classification Token


If you look at the VIT transformer diagram again, you will notice that a classification (CLS) token is passed into the model and only this output is then utilised in the classification head to predict the class of the image. This is a common trick as the learnt parameters of the CLS token can be fine-tuned alongside the weights of the model. Combined with the multi-head self-attention layers, the classification token will be influenced by all other inputs and can be thought of as a summary.

<center><img alt="VIT Transformer Diagram" src="./media/vit_ex.png" style="max-height: 300px;"></center>

In this section, we will look at implementing the CLS token and training the best performing transformer model with the CLS token.

**TASK 9.1:** Define a CLS token within the constructor as an [`nn.Parameter`](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html). You can do this by modifying the code below which creates a learnable parameter in your model:

```
nn.Parameter(torch.randn(<args>))
```

**TASK 9.2:** In the forward method of your model, prepend your CLS token to the input features *after* they have been turned into patches. **Note:** you will need to repeat your CLS token so that the dimensions match, *i.e.* your CLS token is of size $(1, 1, D)$ and your inputs are of size $(B, 16, D)$ and you want a result of $(B, 17, D)$. You can use `[torch.Tensor.Expand](https://pytorch.org/docs/stable/generated/torch.Tensor.expand.html)` and `[torch.cat](https://pytorch.org/docs/stable/generated/torch.cat.html)` to do this.

Afterwards, run your model to see how performance/number of parameters compares to without using the CLS token.

## Removing Global Pooling

Paying closer attention to the VIT transformer diagram above, we can see that only the output from the CLS token is used in the classification head. We can do the same by indexing the output of final encoder layer using `x[:,0,:]` or similar. **NOTE:** make sure you choose the output that matches the CLS token!

**TASK 10:** In the forward method of your method, use only the output corresponding to the CLS token as input to the classification head.

Run your model to check its performance and compare to the global pooling above.

---
# Adding Positional Embeddings

The final part that's missing from our implementation and the VIT model are the positional embeddings. Recall from the lecture, these are added to the input and help the model differentiate the order of the sequence: $f_{0,i}=e(i) + x_i$ where $x_i$ is the $i$th input, $f$ the input to your first layer and $e$ is a function the returns the $i$th positional embedding.

We'll focus on the learnt positional embeddings, which require us to learn $S$ different $D$ length embeddings which we will add to our input. The first question you may ask is what is $S$? This will be the *maximum* length of the sequence that will be inputted, you can work this out based on the image sizes as well as the patch sizes (don't forget about your CLS token too!)

**TASK 11.1:** Within the constructor for `CIFAR_Transformer`, create and randomly initialise the positional embeddings as an `nn.Parameter`.

Once these have been created, the next step is to add them to our input after it has been patchified. Similar to the CLS token, we will need to repeat this across the batch - think about why we *don't* want to learn a $B \times S \times D$ size set of positional embeddings.

**Task 11.2:** In the forward pass, add the positional embeddings to the output of the patchify function following the VIT diagram above (remember we don't have the linear projectiion).

Afterward, run the model and check to see how the performance changes with the positional embeddings.

___

# The State-of-the-Art??

This is the end of all the tasks within the lab, but there is much left to explore and play with for the transformer. You may be interested in modifying the patch size, adding in the linear projection layer, increasing the depth, or modifying the sizes of the feed-forward networks within each encoder layer. Of course there are a whole host of hyperparameters to tune. The lessons learnt over the past two labs can also be applied here, e.g. does data augmentation help the model?

With that said, this takes us to the end of Lab 5, and the last learnt lab for the unit. From humble beginnings of classifying irises with a 2 layer fully-connected network, we have now created a model for CIFAR-10 which is based on the *state-of-the-art*.